# ProjectNotebook2Merged: T5-ParaDetox vs XDetox Pipeline Comparison

This notebook compares two approaches to text detoxification:
1. **T5-ParaDetox**: Supervised fine-tuning of T5-base on ParaDetox dataset
2. **XDetox**: Product-of-Experts (DecompX) approach using base/expert/anti-expert models

Both pipelines are evaluated on multiple test datasets with comprehensive metrics.

## Section 1: Setup & Dependencies

### Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Dependencies

**Approach**: Use Colab's pre-installed packages (same as ProjectNotebook1).

This matches what ProjectNotebook1 actually used in November 2024:
- `transformers` 4.57.1+ (Colab's latest)
- `datasets` 4.0.0+ (Colab's latest)
- No version pinning needed - Colab maintains compatible versions

After running this cell, go to **Runtime → Restart Runtime** to ensure clean imports.

In [4]:
# Install required packages (use Colab's pre-installed versions where possible)
# This matches ProjectNotebook1's approach: let Colab provide compatible versions

# Core packages (likely already installed in Colab)
!pip install -q transformers torch datasets

# XDetox-specific dependencies
!pip install -q sacremoses ftfy nltk

# Evaluation and training packages
!pip install -q evaluate sacrebleu
!pip install -q sentence-transformers
!pip install -q pandas numpy scikit-learn matplotlib
!pip install -q accelerate -U

# Evaluation metrics
!pip install -q rouge_score
!pip install -q bert-score

print("=" * 70)
print("✓ All packages installed")
print("=" * 70)

# Verify all critical imports for both pipelines
print("\n🔍 Verifying package compatibility...\n")

try:
    # Check transformers and datasets versions
    import transformers
    import datasets
    print(f"✓ transformers {transformers.__version__}")
    print(f"✓ datasets {datasets.__version__}")

    # Check XDetox dependencies
    import sacremoses
    import ftfy
    import nltk
    print(f"✓ sacremoses {sacremoses.__version__}")
    print(f"✓ ftfy installed")
    print(f"✓ nltk installed")

    # Check T5-ParaDetox requirements (Trainer import)
    from transformers import (
        Trainer,
        TrainingArguments,
        T5Tokenizer,
        T5ForConditionalGeneration
    )
    print(f"✓ Trainer imports successfully")
    print(f"✓ T5 models available")

    # Check XDetox requirements (apply_chunking_to_forward)
    from transformers.modeling_utils import apply_chunking_to_forward
    print(f"✓ apply_chunking_to_forward available (XDetox needs this)")

    # Check evaluation requirements
    from sentence_transformers import SentenceTransformer
    from evaluate import load
    print(f"✓ SentenceTransformer available")
    print(f"✓ evaluate library available")

    print("\n" + "=" * 70)
    print("🎉 SUCCESS! All dependencies are compatible.")
    print("Both T5-ParaDetox and XDetox pipelines should work correctly.")
    print("=" * 70)

except ImportError as e:
    print(f"\n❌ Import failed: {e}")
    print("\n⚠️  Troubleshooting:")
    print("  1. Go to Runtime → Restart Runtime")
    print("  2. Re-run this cell")
    print("  3. If error persists, check which specific import failed above")
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")

✓ All packages installed

🔍 Verifying package compatibility...

✓ transformers 4.37.2
✓ datasets 2.19.0
✓ sacremoses 0.1.1
✓ ftfy installed
✓ nltk installed

❌ Unexpected error: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)


### Import Libraries

In [5]:
import os
import sys
import re
import json
import shutil
import math
from pathlib import Path
from subprocess import run, PIPE

import torch
import numpy as np
import pandas as pd
import nltk
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

# Import transformers (same as ProjectNotebook1)
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TrainingArguments,
    Trainer
)
from torch.utils.data import Dataset
from sentence_transformers import SentenceTransformer
from evaluate import load

# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"

print("✓ Libraries imported")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

### Setup NLTK

In [ ]:
nltk.download("punkt", quiet=True)
try:
    nltk.download("punkt_tab", quiet=True)
except Exception:
    pass
print("✓ NLTK ready")

### Configure Paths

**IMPORTANT**: Update these paths based on your Google Drive structure!

In [ ]:
# ========================================
# CONFIGURE THESE PATHS FOR YOUR SETUP
# ========================================

# Path to your project folder (contains ParaDetox data and model checkpoints)
# PLACEHOLDER - Update this!
PROJECT_BASE_PATH = "/content/drive/MyDrive/ds266/w266 - Project"

# Path to XDetox repository (contains rewrite/, evaluation/, datasets/ folders)
# PLACEHOLDER - Update this!
XDETOX_DIR = "/content/drive/MyDrive/ds266/w266 - Project/XDetox"

# ========================================
# Verify paths exist
# ========================================
assert os.path.isdir(PROJECT_BASE_PATH), f"PROJECT_BASE_PATH not found: {PROJECT_BASE_PATH}"
assert os.path.isdir(XDETOX_DIR), f"XDETOX_DIR not found: {XDETOX_DIR}"

# Setup XDetox environment
HF_CACHE = os.path.join(XDETOX_DIR, "cache")
os.environ["TRANSFORMERS_CACHE"] = HF_CACHE
if XDETOX_DIR not in sys.path:
    sys.path.append(XDETOX_DIR)

# Verify XDetox repo structure
for d in ["rewrite", "evaluation", "datasets"]:
    assert os.path.isdir(os.path.join(XDETOX_DIR, d)), f"Missing XDetox folder: {d}"

print("✓ PROJECT_BASE_PATH:", PROJECT_BASE_PATH)
print("✓ XDETOX_DIR:", XDETOX_DIR)
print("✓ HF_CACHE:", HF_CACHE)
print("✓ All paths verified")

### Dataset Configuration

These are the available test datasets for both pipelines.

In [ ]:
# XDetox dataset configurations (from XDetox_Pipeline.ipynb)
XDETOX_DATASETS = {
    "paradetox": {
        "data_path": "./datasets/paradetox/test_toxic_parallel.txt",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.75,
        "temperature": 2.5,
    },
    "microagressions_test": {
        "data_path": "./datasets/microagressions/test.csv",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.25,
        "temperature": 2.5,
    },
    "sbf_test": {
        "data_path": "./datasets/sbf/sbftst.csv",
        "rep_penalty": 1.5,
        "alpha_a": 1.5,
        "alpha_e": 5.0,
        "temperature": 2.9,
    },
    "dynabench_test": {
        "data_path": "./datasets/dynabench/db_test.csv",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.75,
        "temperature": 2.5,
    },
    "jigsaw_toxic": {
        "data_path": "./datasets/jigsaw_full_30/test_10k_toxic.txt",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.75,
        "temperature": 2.5,
    },
    "appdia_original": {
        "data_path": "./datasets/appdia/original-annotated-data/original-test.tsv",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.75,
        "temperature": 2.5,
    },
    "appdia_discourse": {
        "data_path": "./datasets/appdia/discourse-augmented-data/discourse-test.tsv",
        "rep_penalty": 1.0,
        "alpha_a": 1.5,
        "alpha_e": 4.75,
        "temperature": 2.5,
    },
}

print(f"✓ {len(XDETOX_DATASETS)} test datasets configured:")
for name in XDETOX_DATASETS.keys():
    print(f"  - {name}")

---
## Section 2: Shared Evaluation Metrics

This evaluator class combines metrics from both pipelines:
- **Style Transfer Accuracy (STA)**: % of outputs classified as non-toxic
- **Content Similarity**: Cosine similarity between original and detoxified embeddings
- **BLEU**: Precision-based n-gram overlap
- **ROUGE-L**: Longest common subsequence
- **BERTScore**: Semantic similarity using BERT embeddings
- **Perplexity**: Language model fluency (GPT-2)
- **Toxicity Score**: Average toxicity of generated text
- **J-Score**: STA × Content Similarity

In [ ]:
class UnifiedDetoxEvaluator:
    """
    Comprehensive evaluator combining metrics from both T5-ParaDetox and XDetox pipelines.
    """
    def __init__(self):
        print("Initializing UnifiedDetoxEvaluator...")

        # Toxicity Classifier (for STA)
        self.toxicity_tokenizer = AutoTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")
        self.toxicity_model = AutoModelForSequenceClassification.from_pretrained("s-nlp/roberta_toxicity_classifier")
        self.toxicity_model.eval()

        # Sentence similarity model
        self.sim_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Metrics from HuggingFace evaluate
        self.bleu = load("sacrebleu")
        self.rouge = load("rouge")
        self.bertscore = load("bertscore")

        # GPT-2 for perplexity (using proper language model)
        self.perplexity_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
        self.perplexity_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
        self.perplexity_model.eval()

        # Set pad token for GPT-2
        if self.perplexity_tokenizer.pad_token is None:
            self.perplexity_tokenizer.pad_token = self.perplexity_tokenizer.eos_token

        print("✓ Evaluator initialized")

    def style_transfer_accuracy(self, texts, batch_size=32):
        """Measure how many texts are classified as non-toxic (STA)"""
        all_predictions = []

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = self.toxicity_tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )

            with torch.no_grad():
                outputs = self.toxicity_model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                # Label 0 = non-toxic, Label 1 = toxic
                non_toxic_probs = predictions[:, 0]
                all_predictions.extend(non_toxic_probs.tolist())

        # STA = percentage classified as non-toxic
        non_toxic_count = sum(1 for p in all_predictions if p > 0.5)
        return non_toxic_count / len(all_predictions)

    def toxicity_scores(self, texts, batch_size=32):
        """Calculate average toxicity probability"""
        all_toxic_probs = []

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = self.toxicity_tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )

            with torch.no_grad():
                outputs = self.toxicity_model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                # Label 1 = toxic
                toxic_probs = predictions[:, 1]
                all_toxic_probs.extend(toxic_probs.tolist())

        return np.mean(all_toxic_probs)

    def content_similarity(self, original_texts, detoxified_texts):
        """Measure semantic similarity using sentence embeddings"""
        original_embeddings = self.sim_model.encode(original_texts, show_progress_bar=False)
        detoxified_embeddings = self.sim_model.encode(detoxified_texts, show_progress_bar=False)

        similarities = []
        for orig, detox in zip(original_embeddings, detoxified_embeddings):
            sim = np.dot(orig, detox) / (np.linalg.norm(orig) * np.linalg.norm(detox))
            similarities.append(sim)

        return np.mean(similarities)

    def calculate_perplexity(self, texts):
        """Calculate average perplexity using GPT-2 language model"""
        perplexities = []

        for text in texts:
            inputs = self.perplexity_tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=512
            )

            with torch.no_grad():
                # Get the loss from the language model
                outputs = self.perplexity_model(**inputs, labels=inputs["input_ids"])
                loss = outputs.loss
                ppl = torch.exp(loss).item()
                perplexities.append(ppl)

        return np.mean(perplexities) if perplexities else float('nan')

    def evaluate_all(self, original_texts, detoxified_texts, reference_texts=None):
        """
        Complete evaluation with all metrics.

        Args:
            original_texts: List of toxic input texts
            detoxified_texts: List of model-generated detoxified texts
            reference_texts: Optional list of ground-truth detoxified texts

        Returns:
            Dictionary of metric scores
        """
        results = {}

        # Style Transfer Accuracy
        results['STA'] = self.style_transfer_accuracy(detoxified_texts)

        # Content Similarity
        results['Content_Similarity'] = self.content_similarity(original_texts, detoxified_texts)

        # Toxicity scores
        results['Toxicity_Gen'] = self.toxicity_scores(detoxified_texts)
        results['Toxicity_Orig'] = self.toxicity_scores(original_texts)

        # Perplexity (fluency)
        results['Perplexity_Gen'] = self.calculate_perplexity(detoxified_texts)

        # If we have ground truth references, compute BLEU/ROUGE/BERTScore
        if reference_texts:
            # BLEU
            bleu_result = self.bleu.compute(
                predictions=detoxified_texts,
                references=[[ref] for ref in reference_texts]
            )
            results['BLEU'] = bleu_result['score'] / 100.0

            # ROUGE
            rouge_result = self.rouge.compute(
                predictions=detoxified_texts,
                references=reference_texts
            )
            results['ROUGE-L'] = rouge_result['rougeL']

            # BERTScore
            bert_result = self.bertscore.compute(
                predictions=detoxified_texts,
                references=reference_texts,
                lang="en"
            )
            results['BERTScore_F1'] = np.mean(bert_result['f1'])
        else:
            # Use original texts as pseudo-references for BLEU/BERTScore
            bleu_result = self.bleu.compute(
                predictions=detoxified_texts,
                references=[[orig] for orig in original_texts]
            )
            results['BLEU'] = bleu_result['score'] / 100.0

            bert_result = self.bertscore.compute(
                predictions=detoxified_texts,
                references=original_texts,
                lang="en"
            )
            results['BERTScore_F1'] = np.mean(bert_result['f1'])

            results['ROUGE-L'] = None  # Can't compute without references

        # J-Score
        results['J-Score'] = results['STA'] * results['Content_Similarity']

        return results

---
## Section 3: T5-ParaDetox Pipeline

This section implements supervised fine-tuning of T5-base on the ParaDetox dataset.

### 3.1: Load ParaDetox Dataset

In [ ]:
# Load the ParaDetox dataset
paradetox_data_path = os.path.join(PROJECT_BASE_PATH, 'paradetox.tsv')
df_paradetox = pd.read_csv(paradetox_data_path, sep='\t')

print(f"✓ Loaded ParaDetox dataset: {len(df_paradetox)} examples")
print(f"✓ Columns: {df_paradetox.columns.tolist()}")
df_paradetox.head(3)

### 3.2: Create Training Data with Augmentation

In [ ]:
def create_augmented_data(df):
    """
    Create training data using ALL available neutral paraphrases.
    This augments the dataset by using neutral1, neutral2, neutral3 separately.
    """
    training_data = []
    for _, row in df.iterrows():
        toxic_text = row['toxic']
        # Use ALL available neutral paraphrases
        for col in ['neutral1', 'neutral2', 'neutral3']:
            if pd.notna(row[col]) and str(row[col]).strip():
                training_data.append({
                    'input': f"detoxify: {toxic_text}",
                    'output': str(row[col]).strip()
                })
    return training_data

# Create augmented training data
paradetox_training_data = create_augmented_data(df_paradetox)
print(f"✓ Total training examples (after augmentation): {len(paradetox_training_data)}")

# Create train/val/test splits (80/10/10)
train_data, temp_data = train_test_split(paradetox_training_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"✓ Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")

### 3.3: Dataset Class for T5

In [ ]:
class DetoxDataset(Dataset):
    """
    PyTorch Dataset for T5 detoxification task.
    Input format: "detoxify: {toxic_text}"
    Output: neutral paraphrase
    """
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Tokenize input
        input_encoding = self.tokenizer(
            item['input'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize target
        target_encoding = self.tokenizer(
            item['output'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

### 3.4: Initialize T5 Model & Training

In [ ]:
# Initialize T5 tokenizer and model
model_name = "t5-base"
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Create PyTorch datasets
train_dataset = DetoxDataset(train_data, t5_tokenizer)
val_dataset = DetoxDataset(val_data, t5_tokenizer)
test_dataset = DetoxDataset(test_data, t5_tokenizer)

print("✓ T5 model and datasets initialized")

### 3.5: Training Configuration

In [ ]:
# Training arguments (default config from ProjectNotebook1)
# NOTE: Training is optional - you can skip to cell 3.8 to load pre-trained checkpoint

training_args = TrainingArguments(
    output_dir='./t5-detox-results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=t5_tokenizer,
)

print("✓ Trainer initialized")

### 3.6: Train the Model

**Note**: Uncomment to actually train. Training takes significant time!

In [ ]:
# Uncomment to train:
# print("Starting training...")
# trainer.train()
# print("✓ Training complete!")

### 3.7: Save Checkpoint

In [ ]:
# Save the fine-tuned model (PLACEHOLDER PATH - update this!)
t5_checkpoint_path = os.path.join(PROJECT_BASE_PATH, 't5-base-detox-model')

# Uncomment to save after training:
# t5_model.save_pretrained(t5_checkpoint_path)
# t5_tokenizer.save_pretrained(t5_checkpoint_path)
# print(f"✓ Model saved to {t5_checkpoint_path}")

### 3.8: Load Checkpoint (for inference)

In [ ]:
# Load the trained model from checkpoint
t5_checkpoint_path = os.path.join(PROJECT_BASE_PATH, 't5-base-detox-model')

print(f"Loading T5 model from {t5_checkpoint_path}...")
t5_tokenizer = T5Tokenizer.from_pretrained(t5_checkpoint_path)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_checkpoint_path)
t5_model.eval()  # Set to evaluation mode

print("✓ T5 model loaded from checkpoint")

### 3.9: Inference Function

In [ ]:
def t5_detoxify_text(text, model, tokenizer, max_length=128):
    """
    Generate detoxified text using T5 model.
    """
    device = next(model.parameters()).device
    input_text = f"detoxify: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=max_length, truncation=True)
    input_ids = input_ids.to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            num_beams=5,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test inference
test_text = "This is a stupid idea"
detoxified = t5_detoxify_text(test_text, t5_model, t5_tokenizer)
print(f"Original: {test_text}")
print(f"Detoxified: {detoxified}")

### 3.10: Helper Function to Load Test Data

In [ ]:
def load_test_dataset(dataset_name, max_examples=None):
    """
    Load test data from various datasets.
    Returns (toxic_texts, reference_texts) where reference_texts may be None.
    """
    if dataset_name not in XDETOX_DATASETS:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    config = XDETOX_DATASETS[dataset_name]
    data_path = os.path.join(XDETOX_DIR, config["data_path"].lstrip("./"))

    toxic_texts = []
    reference_texts = []

    # Handle different file formats
    if data_path.endswith('.txt'):
        # Plain text file (no references)
        with open(data_path, 'r') as f:
            toxic_texts = [line.strip() for line in f if line.strip()]
        reference_texts = None

    elif data_path.endswith('.csv'):
        df = pd.read_csv(data_path)
        # Try to find toxic and reference columns
        if 'text' in df.columns:
            toxic_texts = df['text'].tolist()
        elif 'toxic' in df.columns:
            toxic_texts = df['toxic'].tolist()
        else:
            toxic_texts = df.iloc[:, 0].tolist()  # First column

        # Look for reference column
        if 'neutral' in df.columns:
            reference_texts = df['neutral'].tolist()
        else:
            reference_texts = None

    elif data_path.endswith('.tsv'):
        df = pd.read_csv(data_path, sep='\t')
        if 'text' in df.columns:
            toxic_texts = df['text'].tolist()
        else:
            toxic_texts = df.iloc[:, 0].tolist()
        reference_texts = None

    # Limit number of examples if specified
    if max_examples:
        toxic_texts = toxic_texts[:max_examples]
        if reference_texts:
            reference_texts = reference_texts[:max_examples]

    return toxic_texts, reference_texts

print("✓ Test data loader defined")

### 3.11: Evaluate T5-ParaDetox on Multiple Datasets

In [ ]:
# Initialize evaluator
evaluator = UnifiedDetoxEvaluator()

# Datasets to evaluate on
datasets_to_eval = ["paradetox", "microagressions_test", "sbf_test", "dynabench_test"]
max_examples_per_dataset = 200  # Adjust based on compute resources

# Store results
t5_results = []

print("\n" + "="*80)
print("T5-PARADETOX EVALUATION")
print("="*80)

for dataset_name in datasets_to_eval:
    print(f"\n[{dataset_name}] Loading test data...")

    try:
        toxic_texts, reference_texts = load_test_dataset(dataset_name, max_examples=max_examples_per_dataset)
        print(f"  ✓ Loaded {len(toxic_texts)} examples")

        # Generate detoxified texts
        print(f"  Generating detoxified outputs...")
        detoxified_texts = []
        for text in tqdm(toxic_texts, desc="  T5 Inference"):
            detox = t5_detoxify_text(text, t5_model, t5_tokenizer)
            detoxified_texts.append(detox)

        # Evaluate
        print(f"  Computing metrics...")
        metrics = evaluator.evaluate_all(toxic_texts, detoxified_texts, reference_texts)
        metrics['dataset'] = dataset_name
        metrics['num_examples'] = len(toxic_texts)
        t5_results.append(metrics)

        print(f"  ✓ Done!")

    except Exception as e:
        print(f"  ✗ Error: {e}")
        continue

print("\n" + "="*80)

### 3.12: Display T5-ParaDetox Results Table

In [ ]:
# Create results DataFrame
t5_results_df = pd.DataFrame(t5_results)

# Reorder columns for better readability
column_order = [
    'dataset', 'num_examples',
    'STA', 'Content_Similarity', 'J-Score',
    'BLEU', 'ROUGE-L', 'BERTScore_F1',
    'Toxicity_Gen', 'Toxicity_Orig', 'Perplexity_Gen'
]
t5_results_df = t5_results_df[[col for col in column_order if col in t5_results_df.columns]]

print("\n" + "="*80)
print("T5-PARADETOX RESULTS")
print("="*80)
print(t5_results_df.to_string(index=False))
print("="*80)

---
## Section 4: XDetox Pipeline (DecompX)

This section runs the XDetox product-of-experts approach with masking and reranking.

### 4.1: XDetox Helper Functions

In [ ]:
# Helper functions from XDetox_Pipeline.ipynb

def _abs_repo_path(rel):
    return os.path.join(XDETOX_DIR, rel.lstrip("./"))

def _ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

def _subset_for_data_type(data_type, data_path, n, out_dir):
    """Create a small subset file matching the expected format."""
    if n is None or n <= 0:
        return data_path

    src = _abs_repo_path(data_path)
    _ensure_dir(out_dir)

    if "microagressions" in data_path:
        df = pd.read_csv(src)
        sub = df.head(n)
        out = os.path.join(out_dir, os.path.basename(src))
        sub.to_csv(out, index=False)
        return out

    if "sbf" in data_path:
        df = pd.read_csv(src)
        sub = df.head(n)
        out = os.path.join(out_dir, os.path.basename(src))
        sub.to_csv(out, index=False)
        return out

    if "dynabench" in data_path:
        df = pd.read_csv(src)
        sub = df.head(n)
        out = os.path.join(out_dir, os.path.basename(src))
        sub.to_csv(out, index=False)
        return out

    if any(k in data_path for k in ["paradetox", "jigsaw"]):
        if data_path.endswith(".txt"):
            with open(src, "r") as f:
                lines = [s.rstrip("\n") for s in f.readlines()]
            out = os.path.join(out_dir, os.path.basename(src))
            with open(out, "w") as g:
                for s in lines[:n]:
                    g.write(s + "\n")
            return out
        elif data_path.endswith(".csv"):
            df = pd.read_csv(src).head(n)
            out = os.path.join(out_dir, os.path.basename(src))
            df.to_csv(out, index=False)
            return out

    if "appdia" in data_path:
        df = pd.read_csv(src, sep="\t").head(n)
        out = os.path.join(out_dir, os.path.basename(src))
        df.to_csv(out, sep="\t", index=False)
        return out

    # Fallback
    out = os.path.join(out_dir, os.path.basename(src))
    shutil.copy(src, out)
    return out

def _parse_run_folder_name(folder_name):
    pattern = r"aa(\d+\.\d+)_ae(\d+\.\d+)_ab(\d+\.\d+)_base(.*)_anti(.*)_expert(.*)_temp(\d+\.\d+)_sample(.*)_topk(\d+)_reppenalty(\d+\.\d+)_filterp(\d+\.\d+)_maxlength(\d+)_topp(\d+\.\d+)"
    m = re.match(pattern, folder_name)
    return bool(m)

def _eval_with_toxicity(base_path, overwrite_eval=False, skip_ref=False, tox_threshold=0.5, tox_batch_size=32):
    """Run evaluation using XDetox's evaluation module."""
    for folder in os.listdir(base_path):
        gen_dir = os.path.join(base_path, folder)
        if not os.path.isdir(gen_dir) or not _parse_run_folder_name(folder):
            continue
        orig_path = os.path.join(gen_dir, "orig.txt")
        gen_path  = os.path.join(gen_dir, "gen.txt")
        out_stats = os.path.join(gen_dir, "gen_stats.txt")
        if not (os.path.exists(orig_path) and os.path.exists(gen_path)):
            continue
        if os.path.exists(out_stats) and not overwrite_eval:
            continue

        env = os.environ.copy()
        env["PYTHONPATH"] = XDETOX_DIR + (":" + env.get("PYTHONPATH","") if env.get("PYTHONPATH") else "")
        cmd = [
            sys.executable, "-m", "evaluation.evaluate_all",
            "--orig_path", orig_path,
            "--gen_path",  gen_path,
            "--tox_threshold", str(tox_threshold),
            "--tox_batch_size", str(tox_batch_size),
        ]
        if skip_ref:
            cmd.append("--skip_ref")
        print("Eval:", " ".join(cmd))
        res = run(cmd, cwd=XDETOX_DIR, env=env, stdout=PIPE, stderr=PIPE, text=True)
        if res.returncode != 0:
            print(res.stdout)
            print(res.stderr)
            res.check_returncode()

def _safe_float(x):
    try:
        return float(x)
    except Exception:
        return float('nan')

def _read_stats_file(path):
    """Read gen_stats.txt into a dict of floats."""
    out = {}
    with open(path, "r") as f:
        for line in f:
            if ":" not in line:
                continue
            k, v = line.strip().split(": ", 1)
            k = k.replace("(skipped)", "").strip().lower()
            out[k] = _safe_float(v)
    return out

def _aggregate_eval_csv(output_folder, data_type, base_out_dir):
    """Aggregate evaluation results across thresholds."""
    rows = []
    for thresh in np.arange(0.15, 0.3, 0.05, dtype=np.float64):
        mask_dir = f"DecompX{abs(thresh):g}" if thresh != 0 else "DecompX0.0"
        base_path = os.path.join(base_out_dir, data_type, mask_dir)
        if not os.path.isdir(base_path):
            continue
        for folder in os.listdir(base_path):
            gen_dir = os.path.join(base_path, folder)
            stats_path = os.path.join(gen_dir, "gen_stats.txt")
            if not os.path.exists(stats_path):
                continue
            s = _read_stats_file(stats_path)
            rows.append({
                "threshold": float(f"{thresh:.2f}"),
                "folder": folder,
                "bertscore":        s.get("bertscore", np.nan),
                "bleu4":            s.get("bleu4", np.nan),
                "perplexity_gen":   s.get("perplexity gen", np.nan),
                "perplexity_orig":  s.get("perplexity orig", np.nan),
                "toxicity_gen":     s.get("toxicity gen", np.nan),
                "toxicity_orig":    s.get("toxicity orig", np.nan),
            })

    if rows:
        cols = [
            "threshold", "folder",
            "bertscore", "bleu4",
            "perplexity_gen", "perplexity_orig",
            "toxicity_gen", "toxicity_orig",
        ]
        df = pd.DataFrame(rows)
        df = df[cols]
        out_csv = os.path.join(base_out_dir, data_type, f"{data_type}.csv")
        df.to_csv(out_csv, index=False)
        print("Wrote summary CSV:", out_csv)
        return df
    else:
        print("No evaluation files found to summarize.")
        return None

print("✓ XDetox helper functions loaded")

### 4.2: XDetox `detoxify()` Function

In [ ]:
def xdetox_detoxify(
    data_type: str = "paradetox",
    output_folder: str = "merged_notebook_run",
    thresholds = (0.20,),  # Single threshold for simplicity
    batch_size: int = 10,
    ranking: bool = True,
    sample: bool = True,
    top_k_gen: int = 50,
    top_p: float = 0.95,
    filter_p: float = 1.0,
    max_length: int = 128,
    alpha_a: float = None,
    alpha_e: float = None,
    alpha_b: float = 1.0,
    temperature: float = None,
    rep_penalty: float = None,
    num_examples: int = 200,
    overwrite_gen: bool = False,
    run_eval: bool = True,
    overwrite_eval: bool = False,
    skip_ref_eval: bool = False,
):
    """
    Run the XDetox pipeline (DecompX with product-of-experts).
    """
    assert data_type in XDETOX_DATASETS, f"Unknown data_type: {data_type}"
    cfg = XDETOX_DATASETS[data_type].copy()

    # Fallbacks from config
    if alpha_a is None: alpha_a = cfg["alpha_a"]
    if alpha_e is None: alpha_e = cfg["alpha_e"]
    if temperature is None: temperature = cfg["temperature"]
    if rep_penalty is None: rep_penalty = cfg["rep_penalty"]

    base_out_dir = os.path.join("data", "dexp_outputs", output_folder)
    abs_base_out_dir = os.path.join(XDETOX_DIR, base_out_dir)
    _ensure_dir(abs_base_out_dir)

    # Create subset
    original_data_path = cfg["data_path"]
    subset_dir = os.path.join(XDETOX_DIR, "datasets", "_subsets", data_type)
    _ensure_dir(subset_dir)
    subset_path = _subset_for_data_type(data_type, original_data_path, num_examples, subset_dir)

    # Run thresholds
    for t in thresholds:
        mask_dir = f"DecompX{abs(t):g}" if t != 0 else "DecompX0.0"
        mix_out_dir = os.path.join(abs_base_out_dir, data_type, mask_dir)
        _ensure_dir(mix_out_dir)

        # Build command
        cmd = [
            sys.executable, "-m", "rewrite.rewrite_example",
            "--output_dir", base_out_dir,
            "--data_type", data_type,
            "--data_path", subset_path.replace(XDETOX_DIR + "/", "./"),
            "--rep_penalty", str(rep_penalty),
            "--alpha_a", str(alpha_a),
            "--alpha_e", str(alpha_e),
            "--temperature", str(temperature),
            "--alpha_b", str(alpha_b),
            "--max_length", str(max_length),
            "--batch_size", str(batch_size),
            "--top_k_gen", str(top_k_gen),
            "--top_p", str(top_p),
            "--filter_p", str(filter_p),
            "--thresh", f"{t:.2f}",
        ]
        if ranking:
            cmd.append("--ranking")
        if sample:
            cmd.append("--sample")
        if overwrite_gen:
            cmd.append("--overwrite_gen")

        print("Run:", " ".join(cmd))

        # Run with better error handling
        result = run(cmd, cwd=XDETOX_DIR, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"\n❌ XDetox command failed with exit code {result.returncode}")
            print(f"\n📋 STDOUT:")
            print(result.stdout)
            print(f"\n📋 STDERR:")
            print(result.stderr)
            raise RuntimeError(f"XDetox pipeline failed for {data_type}")

        # Optional evaluation
        if run_eval:
            base_path = os.path.join(abs_base_out_dir, data_type, mask_dir)
            _eval_with_toxicity(
                base_path,
                overwrite_eval=overwrite_eval,
                skip_ref=skip_ref_eval,
                tox_threshold=0.5,
                tox_batch_size=32
            )

    # Summarize
    if run_eval:
        return _aggregate_eval_csv(output_folder, data_type, os.path.join(XDETOX_DIR, "data", "dexp_outputs", output_folder))

    return None

print("✓ XDetox detoxify function defined")

### 4.3: Run XDetox on Multiple Datasets

In [ ]:
# Datasets to evaluate
xdetox_datasets_to_eval = ["paradetox", "microagressions_test", "sbf_test", "dynabench_test"]
xdetox_num_examples = 200

# Store aggregated results
xdetox_results = {}

print("\n" + "="*80)
print("XDETOX (DecompX) EVALUATION")
print("="*80)

for dataset_name in xdetox_datasets_to_eval:
    print(f"\n[{dataset_name}] Running XDetox pipeline...")

    try:
        result_df = xdetox_detoxify(
            data_type=dataset_name,
            output_folder="merged_notebook_run",
            thresholds=(0.20,),  # Single threshold
            batch_size=8,
            ranking=True,
            sample=True,
            num_examples=xdetox_num_examples,
            run_eval=True,
            overwrite_eval=False,
            skip_ref_eval=False
        )

        if result_df is not None:
            xdetox_results[dataset_name] = result_df
            print(f"  ✓ {dataset_name} complete!")
        else:
            print(f"  ✗ No results generated for {dataset_name}")

    except Exception as e:
        print(f"  ✗ Error: {e}")
        continue

print("\n" + "="*80)

### 4.4: Display XDetox Results Table

In [ ]:
print("\n" + "="*80)
print("XDETOX (DecompX) RESULTS")
print("="*80)

for dataset_name, df in xdetox_results.items():
    print(f"\n[{dataset_name}]")
    print(df.to_string(index=False))
    print("-" * 80)

print("="*80)

---
## Section 5: Summary & Comparison

Compare the two approaches side-by-side.

In [ ]:
print("\n" + "="*80)
print("FINAL COMPARISON: T5-ParaDetox vs XDetox")
print("="*80)

print("\n--- T5-PARADETOX RESULTS ---")
if len(t5_results) > 0:
    print(t5_results_df.to_string(index=False))
else:
    print("No T5 results available.")

print("\n--- XDETOX RESULTS ---")
if len(xdetox_results) > 0:
    for dataset_name, df in xdetox_results.items():
        print(f"\n[{dataset_name}]")
        print(df.to_string(index=False))
else:
    print("No XDetox results available.")

print("\n" + "="*80)
print("✓ Evaluation complete!")
print("="*80)

---
## Notes

### Key Differences Between Approaches:

**T5-ParaDetox:**
- Single fine-tuned T5-base model
- Supervised training on parallel toxic/neutral pairs
- Simple inference (beam search)
- Fast and straightforward

**XDetox (DecompX):**
- Product-of-Experts with 3 models (base, expert, anti-expert)
- Masking-based approach with threshold tuning
- Sampling + reranking for diversity
- More complex but potentially more controllable

### Evaluation Metrics:
- **STA**: Higher = more non-toxic outputs
- **Content Similarity**: Higher = better preservation of meaning
- **J-Score**: STA × Similarity (balance between detoxification and preservation)
- **BLEU/ROUGE/BERTScore**: Measure against references (when available)
- **Perplexity**: Lower = more fluent text
- **Toxicity**: Lower = better detoxification